In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import os
import javalang
from javaprep import *
from edge_index import edges
import os
from loaders import get_loaders, get_trees
import matplotlib.pyplot as plt
import random
from datetime import datetime
from datetime import date
import csv
import pandas as pd
import torch.optim as optim
import numpy as np
from sklearn import metrics
import statistics

In [2]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch

class GCN(torch.nn.Module):
    def __init__(self, seed):
        torch.manual_seed(seed)
        super(GCN, self).__init__()
        self.conv1 = GCNConv(50, 64)
        self.conv2 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, 64)
        self.lin = Linear(64, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        
        
        return x

In [3]:
def learn(train, test, cwe):
    
    seed = random.randint(1,10000)
    model = GCN(seed)
    learning_rate = 0.01
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = loss_function
    current_time = (datetime.now()).strftime("%H:%M:%S")
    current_date = (date.today()).strftime("%d/%m/%Y")
    epochs, losses, test_accs, train_accs, precisions, recalls, fscores = [], [], [], [], [], [], []
    score_report = {
        'Test Acc': [], 
        'Losses': [],
        'FScores': [],
        'Recalls': [],
        'Precisions': [],
        'CWE': cwe, 
        'Model': str(model).replace('\n', ' ').replace('\t', ' '), 
        'Date': current_date, 
        'Time': current_time, 
        'Seed': seed, 
        'Learning Rate': learning_rate, 
        'Loss Function': loss_function
    }
    epochs_no_improve = 0
    saved = False
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.08)
    for epoch in range(40):
        print("Epoch: ", epoch, end='\r')
        track = []
        for data in train:   
            optimizer.zero_grad()  
            out = model(data.x, data.edge_index, data.batch)  
            loss = criterion(out, data.y)
            pred = out.argmax(dim=1)
            loss.backward()  
            optimizer.step()
            track.append(round(metrics.accuracy_score(data.y.tolist(), pred.tolist()), 3))
        train_accs.append(statistics.mean(track))
        scheduler.step()
        track_a, track_p, track_r, track_f = [],[],[],[]
        for quiz in test:
            out = model(quiz.x, quiz.edge_index, quiz.batch) 
            pred = out.argmax(dim=1)
            track_a.append(metrics.accuracy_score(quiz.y.tolist(), pred.tolist()))
            track_p.append(metrics.precision_score(quiz.y.tolist(), pred.tolist(), zero_division=0))
            track_r.append(metrics.recall_score(quiz.y.tolist(), pred.tolist(), zero_division=0))
            track_f.append(metrics.f1_score(quiz.y.tolist(), pred.tolist(), zero_division=0))
        test_accs.append(statistics.mean(track_a))
        precisions.append(statistics.mean(track_p))
        recalls.append(statistics.mean(track_r))
        fscores.append(statistics.mean(track_f))
        epochs.append(epoch)
        losses.append(loss)
        if epoch+1 in [1,3,5,10,20,40]:
            score_report['Test Acc'].append(statistics.mean(track_a))
            score_report['Losses'].append(round(loss.item(), 3))
            score_report['FScores'].append(statistics.mean(track_f))
            score_report['Recalls'].append(statistics.mean(track_r))
            score_report['Precisions'].append(statistics.mean(track_p))
        
        
    plt.figure(figsize=(20,10))
    x_ticks = np.arange(0, 40, 5)
    plt.xticks(x_ticks)
    y_ticks = np.arange(0, 1, 0.1)
    plt.yticks(y_ticks)
    plt.plot(epochs, losses, label='train loss', color='darkviolet', linewidth=2)
    plt.plot(epochs, train_accs, label='train acc', color='gold', linewidth=2)
    plt.plot(epochs, test_accs, label='test acc', color='forestgreen', linewidth=2)
    plt.plot(epochs, precisions, label='precision', color='dodgerblue', linewidth=2)
    plt.plot(epochs, recalls, label='recall', color='gray', linewidth=2)
    plt.plot(epochs, fscores, label='f-score', color='crimson', linewidth=2)
    plt.legend(prop={'size': 20})
    plt.savefig('../pngs/OWASP-'+str(cwe)+'-'+str(current_date).replace('/','-')+'-'+str(current_time)+'.png')
    plt.clf()
    print()
                                             
    return score_report, model

In [4]:
vocabdict, treedict = get_trees()

creating asts...
creating trees...


In [7]:
vulnerabilities = [22,78,79,89,90,327,328,330,501,614,643]
for cwe in vulnerabilities:
    try:
        print("Learning on CWE"+str(cwe)+"...")
        current_time = (datetime.now()).strftime("%H:%M:%S")
        current_date = (date.today()).strftime("%d/%m/%Y")
        models = []
        ending_acc = []
        sps = []
        for i in range(3):
            train, test = get_loaders(cwe, "owasp", vocabdict, treedict)
            score_report, model = learn(train, test, cwe)
            ending_acc.append(score_report['Test Acc'][-1])
            models.append(model)
            sps.append(score_report)
            if max(ending_acc) == 1:
                break
        best_run = ending_acc.index(max(ending_acc))
        score_report = sps[best_run]
        model = models[best_run]
        torch.save(model.state_dict(), '../models/OWASP-'+str(cwe)+'-'+(current_date).replace('/','')+'-'+current_time.replace('/',''))
        row = [
            score_report['CWE'],
            score_report['Date'],
            score_report['Time'],
            score_report['Model'],
            score_report['Seed'],
            score_report['Learning Rate'],
            score_report['Loss Function'],
        ]
        for epoch in zip(score_report['Test Acc'], score_report['Precisions'], score_report['FScores'], score_report['Recalls']):
            for item in epoch:
                row.append(item)
        print(row)       
        with open('../owasp_score_report.csv','a') as fd:
            writer = csv.writer(fd)
            writer.writerow(row)
    except Exception as e:
        print(e)
        print("failed cwe "+str(cwe))
        pass
print("Done")

Learning on CWE22...
22
training word2vec...
creating graphs..
balancing dataset (pass 1)
Number of good methods:  1706  Number of bad methods:  1706
splitting dataset
Epoch:  39
22
training word2vec...
creating graphs..
balancing dataset (pass 1)
Number of good methods:  1706  Number of bad methods:  1706
splitting dataset
Epoch:  39
22
training word2vec...
creating graphs..
balancing dataset (pass 1)
Number of good methods:  1706  Number of bad methods:  1706
splitting dataset
Epoch:  39
[22, '28/12/2020', '09:43:16', 'GCN(   (conv1): GCNConv(50, 64)   (conv2): GCNConv(64, 64)   (conv3): GCNConv(64, 64)   (lin): Linear(in_features=64, out_features=2, bias=True) )', 9945, 0.01, CrossEntropyLoss(), 0.771823347107438, 0.8413872446626457, 0.7189563777414097, 0.6415943676128731, 0.8327737603305785, 0.8821337753155936, 0.8066853751459788, 0.7538165611712614, 0.8766787190082644, 0.8708726549195871, 0.8681865929444778, 0.8799710578066896, 0.9034090909090909, 0.8426316967268276, 0.90247281709

/home/gparte1/.local/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



[328, '28/12/2020', '11:27:20', 'GCN(   (conv1): GCNConv(50, 64)   (conv2): GCNConv(64, 64)   (conv3): GCNConv(64, 64)   (lin): Linear(in_features=64, out_features=2, bias=True) )', 1008, 0.01, CrossEntropyLoss(), 0.5099537037037037, 0.49216637272192826, 0.5927233367206577, 0.7627461690929183, 0.4648148148148148, 0.4695041816009558, 0.6259584464136666, 0.9573481214348087, 0.4756944444444444, 0.44784567284567284, 0.46307155394376487, 0.49524625414718293, 0.5314814814814814, 0.5526455026455026, 0.23232121335836506, 0.15399562884083007, 0.5344907407407408, 0.5087126542548831, 0.6404357790695977, 0.8860820194024529, 0.4965277777777778, 0.4850953254209959, 0.6075204918301591, 0.833280253171894]
Learning on CWE330...
330
training word2vec...
creating graphs..
balancing dataset (pass 1)
Number of good methods:  2028  Number of bad methods:  2028
splitting dataset
Epoch:  39
[330, '28/12/2020', '11:36:26', 'GCN(   (conv1): GCNConv(50, 64)   (conv2): GCNConv(64, 64)   (conv3): GCNConv(64, 64) 

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

...

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>